In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../data/raw/E0.csv')
df.head()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,11/08/2023,20:00,Burnley,Man City,0,3,A,0,2,...,2.28,1.50,1.95,1.98,1.95,1.97,NaN,NaN,1.92,1.95
1,E0,12/08/2023,12:30,Arsenal,Nott'm Forest,2,1,H,2,0,...,2.63,-2.00,1.95,1.98,1.93,1.97,2.01,2.09,1.95,1.92
2,E0,12/08/2023,15:00,Bournemouth,West Ham,1,1,D,0,0,...,2.12,0.00,2.02,1.91,2.01,1.92,2.06,1.96,1.96,1.91
3,E0,12/08/2023,15:00,Brighton,Luton,4,1,H,1,0,...,2.48,-1.75,2.01,1.92,2.00,1.91,2.14,1.93,2.00,1.86
4,E0,12/08/2023,15:00,Everton,Fulham,0,1,A,0,0,...,1.71,-0.25,2.06,1.87,2.04,1.88,2.08,1.99,1.98,1.88


In [6]:
# Home team records
home_df = df[['Date', 'HomeTeam', 'FTHG', 'FTAG', 'FTR']].copy()
home_df.columns = ['Date', 'Team', 'GoalsFor', 'GoalsAgainst', 'Result']

home_df['Points'] = home_df['Result'].map({
    'H': 3,
    'D': 1,
    'A': 0
})

# Away team records
away_df = df[['Date', 'AwayTeam', 'FTAG', 'FTHG', 'FTR']].copy()
away_df.columns = ['Date', 'Team', 'GoalsFor', 'GoalsAgainst', 'Result']

away_df['Points'] = away_df['Result'].map({
    'A': 3,
    'D': 1,
    'H': 0
})

# Combine both
team_match_df = pd.concat([home_df, away_df], ignore_index=True)

team_match_df.head()


,Date,Team,GoalsFor,GoalsAgainst,Result,Points
0,11/08/2023,Burnley,0,3,A,0
1,12/08/2023,Arsenal,2,1,H,3
2,12/08/2023,Bournemouth,1,1,D,1
3,12/08/2023,Brighton,4,1,H,3
4,12/08/2023,Everton,0,1,A,0


In [7]:
team_match_df['Team'].nunique()


20

In [8]:
team_match_df.to_csv(
    "../data/processed/team_match_level.csv",
    index=False
)


In [9]:
team_summary = (
    team_match_df
    .groupby('Team')
    .agg(
        Matches=('Team', 'count'),
        GoalsFor=('GoalsFor', 'sum'),
        GoalsAgainst=('GoalsAgainst', 'sum'),
        Points=('Points', 'sum')
    )
    .reset_index()
)

team_summary


,Team,Matches,GoalsFor,GoalsAgainst,Points
0,Arsenal,38,91,29,89
1,Aston Villa,38,76,61,68
2,Bournemouth,38,54,67,48
3,Brentford,38,56,65,39
4,Brighton,38,55,62,48
5,Burnley,38,41,78,24
6,Chelsea,38,77,63,63
7,Crystal Palace,38,57,58,49
8,Everton,38,40,51,48
9,Fulham,38,55,61,47


In [10]:
team_summary['GoalDifference'] = (
    team_summary['GoalsFor'] - team_summary['GoalsAgainst']
)

team_summary['PointsPerMatch'] = (
    team_summary['Points'] / team_summary['Matches']
)


In [11]:
team_summary.head(
     10
)


,Team,Matches,GoalsFor,GoalsAgainst,Points,GoalDifference,PointsPerMatch
0,Arsenal,38,91,29,89,62,2.342105
1,Aston Villa,38,76,61,68,15,1.789474
2,Bournemouth,38,54,67,48,-13,1.263158
3,Brentford,38,56,65,39,-9,1.026316
4,Brighton,38,55,62,48,-7,1.263158
5,Burnley,38,41,78,24,-37,0.631579
6,Chelsea,38,77,63,63,14,1.657895
7,Crystal Palace,38,57,58,49,-1,1.289474
8,Everton,38,40,51,48,-11,1.263158
9,Fulham,38,55,61,47,-6,1.236842


In [12]:
team_summary.to_csv(
    "../data/processed/team_summary.csv",
    index=False
)
